# How to Run?

Easy! Run the cells one by one, and read any instructions as you go along.
All you have to do is ensure that:
a) The number of students matches up with the number of students that the script finds.
b) Your E-Mail and Message details are filled out in the relevant cell.

### Prerequisites
You'll need nbconvert if you don't have it.

In [ ]:
! pip install nbconvert


### Imports
All of these outside nbconvert/nbformat should come with your installation of python. If not, it's a good idea to pip install them. 

In [5]:
import os
import re
import codecs
import smtplib
import nbformat
from nbconvert import HTMLExporter
from email import encoders
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Code
Student Class

In [6]:
class Student:
    """A class to represent a student.
    
    Attributes:
    vunet_id (str): The vunet id of the student.
    ipynb_file (str): The path to the student's ipynb file.
    converted_file (str): The path to the student's converted file.
    parent_folder (str): The parent folder of the student's ipynb file.
    email_address (str): The email address of the student.
    """
    def __init__(self, vunet_id: str, ipynb_file: str = None, converted_file: str = None) -> None:
        """Initializes a student object.
        
        Args:
        vunet_id (str): The vunet id of the student.
        ipynb_file (str): The path to the student's ipynb file.
        converted_file (str): The path to the student's converted file.
        """
        
        self.vunet_id = vunet_id
        self.ipynb_file = ipynb_file
        self.converted_file = converted_file
        self.parent_folder = os.path.dirname(self.ipynb_file) if self.ipynb_file else None

        # Turns out, you don't need to scavenge the student vu email
        # you can just use the vunet id, and it's treated the same
        self.email_address = self.vunet_id + "@student.vu.nl" if self.vunet_id else None

    def __eq__ (self, other) -> bool:
        return self.vunet_id == other.vunet_id
    
    def __repr__(self) -> str:
        return self.vunet_id

Yes, this is a class without methods. Peak OOP.

In [16]:
class TA:
    """Defines info stored about the person sending the emails.
    
    Attributes:
    email_address (str): The email address of the TA.
    password (str): The password of the TA.
    assessment (str): The name of the assessment (e.g. "Midterm Exam" or "Assignment 1").
    """
    def __init__(self, email_address: str, password: str, assessment: str):
        assert(email_address != "EMAIL" and password != "PASSWORD"), "Dude... really? Fill in the email and password."
        assert(assessment != "ASSIGNMENT NR"), "Fill in the assessment name. (ASSIGNMENT_NR)"
        self.email_address = email_address
        self.password = password
        self.assessment = assessment

Main workhorse code. I've tried to actually make it readable in this iteration, but most methods still need docstrings and all that jazz.

In [ ]:
class Converter:
    """A class to convert ipynb files to HTML files and send them to students.

    ASSUMPTIONS:
    - The vunet id is the first occurence of a 3-letter word followed by a 3-digit number.
    - The vunet id is unique.
    - All files are stored in either:
        - The parent directory of the current working directory (i.e. the folder this file is in)
        - A subdirectory the parent directory that is not this folder.
    
    Attributes:
    vunet_finding_mode (str): The mode used to find vunet ids. Currently only "SCRAPE" is supported.
    dir_path (str): The path to the directory containing the ipynb files. if not specified, the parent dir of the current working directory is used.
    valid_students (list[Student]): A list of students with valid vunet ids.
    invalid_students (list[Student]): A list of students with invalid vunet ids.
    """
    
    # Constants
    BANNED_FOLDERS = ["__pycache__", ".ipynb_checkpoints", "ipynb-to-html-converter"]
    BANNED_VUNET_IDS = ["abc123"]  
    
    def __init__(self, id_mode: str, dir_path: str = os.path.dirname(os.getcwd())):
        """Initializes a Converter object.

        Args:
            id_mode (str): The mode used to find vunet ids. Currently only "SCRAPE" is supported.
            dir_path (str, optional): Dir containing the files. Defaults to os.path.dirname(os.getcwd()).
        """
        assert id_mode in ["SCRAPE"]
        assert os.path.isdir(dir_path)
        
        self.vunet_finding_mode = id_mode
        self.dir_path = dir_path
        self.valid_students: list[Student] = []
        self.invalid_students: list[Student] = []
        
    def parse_files(self) -> None:
        """Parses the files in the directory to find vunet ids. Modifies the valid_students and invalid_students lists.
        
        3 steps:
        1. Clear the students lists.
        2. Get the list of directories to parse (i.e. parent of this folder, and all direct children of it, excluding this folder).
        3. If any of these has an ipynb file, a student is added for said file. 
            If it has a valid VUnet, the student gets added to valid_students.      (Student object with vunet_id, ipynb_file, email, and parent_folder)
            If it has an invalid VUnet, the student gets added to invalid_students. (Student object with ipynb_file and parent_folder)
        """
        self.__clear_students()
        dirs = self.__get_dirs_to_parse()
        self.__find_vunet_ids(dirs)
        
    def convert_files(self) -> None:
        """For each student, converts the ipynb file to an HTML file. Modifies the converted_file attribute of each student."""
        all_students = self.valid_students + self.invalid_students
        total_students = len(all_students)
        print(f"Converting {total_students} files to HTML")
        for i, student in enumerate(all_students):
            self.__convert_ipynb_to_html(student)
            self.__visualize_progress(i+1, total_students)
    
    def send_emails(self, ta: TA, message: str, test_mode: bool = False, go_to_vunet_id: str = None) -> None:
        """Sends emails to the students with the converted files attached. Modifies nothing."""
        for i, student in enumerate(self.valid_students):
            if go_to_vunet_id and student.vunet_id != go_to_vunet_id:
                continue
            self.__send_email(ta, message, student, test_mode)
            
            if test_mode:
                break
            
            self.__visualize_progress(i+1, len(self.valid_students))
    
    def __send_email(self, ta: TA, message: str, student: Student, test_mode: bool = False):
        to_address = student.email_address if not test_mode else ta.email_address
        mail = MIMEMultipart()

        mail["From"] = ta.email_address
        mail["To"] = to_address
        mail["Cc"] = ta.email_address
        
        mail["Subject"] = f'{ta.assessment} Feedback and Results'
        mail.attach(MIMEText(message, 'plain'))
        
        with open(student.converted_file, 'rb') as f:
            attachment = f.read()
            part = MIMEBase('application', "octet-stream")
            part.set_payload(attachment)
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', "attachment; filename= %s" % os.path.basename(student.converted_file))
            mail.attach(part)
        
        server = smtplib.SMTP("mails.vu.nl", port=587)
        server.ehlo()
        server.starttls()
        server.ehlo()
        server.login(ta.email_address, ta.password)
        server.send_message(mail)
        server.quit()
    
    def __clear_students(self):
        self.valid_students = []
        self.invalid_students = []
    
    def __convert_ipynb_to_html(self, student: Student):
        if self.__check_if_notebook_exists(student):
            return
        
        with open(student.ipynb_file) as f:
            nb = nbformat.read(f, as_version=4)

        # Convert the notebook to HTML
        html_exporter = HTMLExporter()
        body, _ = html_exporter.from_notebook_node(nb)

        # Write the HTML to a file
        html_file_name = student.vunet_id + ".html" if student.vunet_id else self.__get_expected_html_file_path(student)
        html_file_path = os.path.join(student.parent_folder, html_file_name)
        with open(html_file_path, 'w') as f:
            f.write(body)
    
    def __check_if_notebook_exists(self, student: Student) -> bool:
        expected_file_path = self.__get_expected_html_file_path(student)
        if student.vunet_id:
            ex_dir = os.path.dirname(student.ipynb_file)
            expected_filename = student.vunet_id + ".html"
            expected_file_path = os.path.join(ex_dir, expected_filename)
        
        if os.path.exists(expected_file_path):
            student.converted_file = expected_file_path
            return True
        return False
            
    def __get_expected_html_file_path(self, student: Student):
        return (student.ipynb_file.removesuffix(".ipynb")) + ".html"
            
        
    def __find_vunet_ids(self, dirs: list[str]):

        if self.vunet_finding_mode == "SCRAPE":
            self.__find_vunet_ids_by_scrape(dirs)

    def __find_vunet_ids_by_scrape(self, dirs: list[str]):
        for subDir in dirs:
            self.__scrape_folder(subDir)
    
    def __scrape_folder(self, folder: str):
        for file in os.listdir(folder):
            if self.__is_valid_target_file(file):
                filepath = os.path.join(folder, file)
                self.__scrape_vunet_id_from_file(filepath)   
                
    def __is_valid_target_file(self, file: str):
        return file.endswith(".ipynb")
    
    def __scrape_vunet_id_from_file(self, file: str):
        f = codecs.open(file.strip('"'), 'r')
        for line in f:
            if "VUnet" in line:
                extracted_id = re.search(r'\b[a-zA-Z]{3}\d{3}\b', line)
                if extracted_id is not None:
                    vunet_id = extracted_id.group().lower()
                    if vunet_id in self.BANNED_VUNET_IDS:
                        continue
                    
                    student = Student(vunet_id = vunet_id, ipynb_file=file)
                    self.valid_students.append(student)
                    break
        else:
            student = Student(None, file)
            self.invalid_students.append(student)
                    
    def __visualize_progress(self, current: int, total: int):
        print(f"Progress: {current}/{total}", end="\r", flush=True)
        
    def __get_dirs_to_parse(self):
        child_dirs : list[str] = [folder for folder in os.listdir(self.dir_path) if os.path.isdir(os.path.join(self.dir_path, folder))]
        dirs_to_parse = [os.path.join(self.dir_path, folder) for folder in child_dirs if folder not in self.BANNED_FOLDERS]
        dirs_to_parse = [self.dir_path] + dirs_to_parse
        return dirs_to_parse
        


## Start Reading the Instructions Now

### Parsing all students
Make sure the total number of students is correct

In [9]:
c = Converter(id_mode = "SCRAPE")
c.parse_files()
print(f"Found {len(c.valid_students) + len(c.invalid_students)} files, of which:")
print(f"{len(c.valid_students)} are valid.")
print(f"{len(c.invalid_students)} did not change their VUnet ID or gave an invalid one.")

Found 393 files, of which:
380 are valid.
13 did not change their VUnet ID or gave an invalid one.


### Convert .ipynb Files to .html
You don't need to do much here other than run the file, but this will take a while

In [12]:
c.convert_files()

Converting 393 files to HTML


KeyboardInterrupt: 

## Sending emails
Replace the values at the top of the box below with valid values. Then run. Might be worth a run with `test_mode = true` to see what students will see when they get a message.

In [ ]:
# Email login details
TA_EMAIL = "EMAIL"
TA_PASSWORD = "PASSWORD"

# For formatting email message content
TA_NAME = "TA NAME"                 # Eg. John Doe
ASSIGNMENT_NR = "ASSIGNMENT NR"     # Eg. Assignment 1, Midterm Exam, etc.

ta = TA(TA_EMAIL, TA_PASSWORD, ASSIGNMENT_NR)
assert(TA_NAME != "TA NAME"), "TA NAME should probably be filled"


# Email message content
message = f'''Dear student,

Please find attached your graded {ASSIGNMENT_NR} with appropriate feedback. Details about the inspection hour will follow soon. In case you have questions, please contact your TA. All responses to this email will be ignored.

Kind regards and have a great day,
{TA_NAME}'''


# when true, only sends one email to the TA
TEST_MODE = True

# when not None, skips ahead in the list of students to this vunet id, then continues as normal
# This is mostly legacy from when sending emails would crash occasionally. Still useful for testing.
GO_TO_VUNET_ID = None 

c.send_emails(ta, message, test_mode=True)

AssertionError: Fill in the assessment name. (ASSIGNMENT_NR)

### Make sure to check these files.
These students gave an invalid vunet id or didn't change it from the default.

In [ ]:
invalid_students = c.invalid_students

print("Dirs to check:")
for student in invalid_students:
    print(f"File:   {os.path.basename(student.ipynb_file)}")
    print(f"In Dir: {student.parent_folder}")

    print("\n")

# Running into issues?
Contact me at t.j.but@student.vu.nl or grampachampa@gmail.com.